In [1]:
## Set up data

In [1]:
from theano.sandbox import cuda
cuda.use('gpu')

%matplotlib inline
import utils; reload(utils)
import math
import six
from sklearn.preprocessing import normalize
from utils import *
from __future__ import division, print_function

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
path = "data/"
batch_size=64

In [3]:
_trainData = pd.read_csv(path+'train.csv').values

In [4]:
print(_trainData.shape)

(42000, 785)


In [5]:
trainData = _trainData
labelData = _trainData[:,0]

In [6]:
print(trainData.shape)
print(labelData.shape)

(42000, 785)
(42000,)


In [7]:
trainData = np.delete(trainData,0,axis=1)

In [9]:
print(trainData.shape)

(42000, 784)


In [8]:
def reshapeTrainData(trainData):
    x =[]
    for data in trainData:
        x.append(np.reshape(data, (28, 28)))
    return np.array(x)

In [9]:
trainData = reshapeTrainData(trainData)

In [12]:
print(trainData.shape)

(42000, 28, 28)


In [10]:
X_train = np.expand_dims(trainData,1)

In [14]:
print(X_train.shape)

(42000, 1, 28, 28)


In [11]:
y_train = onehot(labelData)

In [16]:
print(y_train.shape)

(42000, 10)


In [12]:
_X_train = X_train;

X_train = _X_train[0:41900,:,:,:]
val_train = _X_train[41900:42001,:,:,:]

In [18]:
print(X_train.shape)
print(val_train.shape)

(41900, 1, 28, 28)
(100, 1, 28, 28)


In [13]:
_y_train =y_train
y_train = _y_train[0:41900,:]
val_y_train =_y_train[41900:42001,:]

In [20]:
print(y_train.shape)
print(val_y_train.shape)

(41900, 10)
(100, 10)


In [21]:
## norm_input

In [14]:
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)

In [15]:
def norm_input(x): return (x-mean_px)/std_px

In [24]:
## dense model

In [25]:
def get_fc_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Flatten(),
        Dense(512, activation='softmax'),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [26]:
#fc = get_fc_model()

In [27]:
#fc.fit(X_train, y_train,validation_data=(val_train, val_y_train),nb_epoch=200, batch_size=64)

In [28]:
## Batchnorm + VGG MODEL

In [22]:
def get_model_bn():
    model = Sequential([
        Lambda(norm_input, input_shape=(1,28,28)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(64,3,3, activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=1),
        Convolution2D(128,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(1024, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [23]:
model = get_model_bn()

In [28]:
model.optimizer.lr= 1e-6

In [31]:
model.fit(X_train, y_train,validation_data=(val_train, val_y_train),nb_epoch=3, batch_size=64)

Train on 41900 samples, validate on 100 samples
Epoch 1/3
41900/41900 [==============================] - 31s - loss: 0.0177 - acc: 0.9946 - val_loss: 0.0014 - val_acc: 1.0000
Epoch 2/3
41900/41900 [==============================] - 31s - loss: 0.0169 - acc: 0.9952 - val_loss: 0.0021 - val_acc: 1.0000
Epoch 3/3
41900/41900 [==============================] - 31s - loss: 0.0178 - acc: 0.9951 - val_loss: 4.5226e-05 - val_acc: 1.0000


In [32]:
model.fit(X_train, y_train,validation_data=(val_train, val_y_train),nb_epoch=3, batch_size=64)

Train on 41900 samples, validate on 100 samples
Epoch 1/3
41900/41900 [==============================] - 31s - loss: 0.0126 - acc: 0.9961 - val_loss: 4.0690e-05 - val_acc: 1.0000
Epoch 2/3
41900/41900 [==============================] - 31s - loss: 0.0128 - acc: 0.9966 - val_loss: 9.8018e-05 - val_acc: 1.0000
Epoch 3/3
41900/41900 [==============================] - 31s - loss: 0.0138 - acc: 0.9965 - val_loss: 2.7444e-04 - val_acc: 1.0000


In [33]:
model.fit(X_train, y_train,validation_data=(val_train, val_y_train),nb_epoch=3, batch_size=64)

Train on 41900 samples, validate on 100 samples
Epoch 1/3
41900/41900 [==============================] - 31s - loss: 0.0121 - acc: 0.9963 - val_loss: 0.0037 - val_acc: 1.0000
Epoch 2/3
41900/41900 [==============================] - 31s - loss: 0.0120 - acc: 0.9967 - val_loss: 0.0699 - val_acc: 0.9800
Epoch 3/3
41900/41900 [==============================] - 31s - loss: 0.0099 - acc: 0.9969 - val_loss: 0.0013 - val_acc: 1.0000


In [34]:
model.fit(X_train, y_train,validation_data=(val_train, val_y_train),nb_epoch=3, batch_size=64)

Train on 41900 samples, validate on 100 samples
Epoch 1/3
41900/41900 [==============================] - 31s - loss: 0.0108 - acc: 0.9969 - val_loss: 0.0164 - val_acc: 0.9900
Epoch 2/3
41900/41900 [==============================] - 31s - loss: 0.0118 - acc: 0.9964 - val_loss: 7.3906e-05 - val_acc: 1.0000
Epoch 3/3
41900/41900 [==============================] - 31s - loss: 0.0103 - acc: 0.9970 - val_loss: 0.0011 - val_acc: 1.0000


In [36]:
model.fit(X_train, y_train,validation_data=(val_train, val_y_train),nb_epoch=10, batch_size=64)

Train on 41900 samples, validate on 100 samples
Epoch 1/10
41900/41900 [==============================] - 31s - loss: 0.0117 - acc: 0.9969 - val_loss: 9.4988e-06 - val_acc: 1.0000
Epoch 2/10
41900/41900 [==============================] - 31s - loss: 0.0087 - acc: 0.9974 - val_loss: 0.0281 - val_acc: 0.9900
Epoch 3/10
41900/41900 [==============================] - 31s - loss: 0.0098 - acc: 0.9972 - val_loss: 0.0953 - val_acc: 0.9900
Epoch 4/10
41900/41900 [==============================] - 31s - loss: 0.0074 - acc: 0.9979 - val_loss: 0.0150 - val_acc: 0.9900
Epoch 5/10
41900/41900 [==============================] - 31s - loss: 0.0065 - acc: 0.9985 - val_loss: 0.0525 - val_acc: 0.9900
Epoch 6/10
41900/41900 [==============================] - 32s - loss: 0.0066 - acc: 0.9981 - val_loss: 0.0484 - val_acc: 0.9900
Epoch 7/10
41900/41900 [==============================] - 31s - loss: 0.0103 - acc: 0.9972 - val_loss: 0.0043 - val_acc: 1.0000
Epoch 8/10
41900/41900 [============================

In [37]:
model.optimizer.lr= 1e-7

In [38]:
model.fit(X_train, y_train,validation_data=(val_train, val_y_train),nb_epoch=10, batch_size=64)

Train on 41900 samples, validate on 100 samples
Epoch 1/10
41900/41900 [==============================] - 32s - loss: 0.0080 - acc: 0.9978 - val_loss: 1.4933e-05 - val_acc: 1.0000
Epoch 2/10
41900/41900 [==============================] - 32s - loss: 0.0049 - acc: 0.9987 - val_loss: 0.0088 - val_acc: 0.9900
Epoch 3/10
41900/41900 [==============================] - 31s - loss: 0.0053 - acc: 0.9985 - val_loss: 1.7929e-05 - val_acc: 1.0000
Epoch 4/10
41900/41900 [==============================] - 31s - loss: 0.0068 - acc: 0.9981 - val_loss: 5.5283e-05 - val_acc: 1.0000
Epoch 5/10
41900/41900 [==============================] - 32s - loss: 0.0041 - acc: 0.9987 - val_loss: 4.6513e-06 - val_acc: 1.0000
Epoch 6/10
41900/41900 [==============================] - 32s - loss: 0.0068 - acc: 0.9981 - val_loss: 8.9367e-05 - val_acc: 1.0000
Epoch 7/10
41900/41900 [==============================] - 31s - loss: 0.0064 - acc: 0.9984 - val_loss: 1.5020e-07 - val_acc: 1.0000
Epoch 8/10
41900/41900 [========

In [39]:
model.save_weights("model.h5")

In [40]:
_testData = pd.read_csv(path+'test.csv').values
testData = reshapeTrainData(_testData)
testData =np.expand_dims(testData,1)
preds = model.predict(testData)
class_labels = np.argmax(preds, axis=1) 
idx = np.arange(class_labels.shape[0]+1)[1:class_labels.shape[0]+1]
subm = np.stack([idx,class_labels], axis=1)
submission_file_name = 'submission5.csv'
np.savetxt(submission_file_name, subm, fmt='%d,%d', header='ImageId,Label', comments='')